# CacheBackedEmbeddings

`Embeddings` có thể được lưu trữ hoặc lưu vào bộ nhớ cache tạm thời để tránh tính toán lại.

Việc lưu vào bộ nhớ cache `embeddings` có thể được thực hiện bằng cách sử dụng `CacheBackedEmbeddings`. Một trình nhúng được hỗ trợ bởi bộ nhớ cache là một trình bao bọc xung quanh một trình nhúng, giúp lưu trữ `embeddings` trong một kho lưu trữ key-value. Văn bản được băm (hashed) và giá trị băm được sử dụng làm khóa trong bộ nhớ cache.

```bash
python install faiss-cpu
```


Check and create the ./cache/ directory for persistent storage.

In [5]:
import os

os.makedirs("./cache/", exist_ok=True)
print(os.path.exists("./cache/"))  # Check if the directory exists
print(os.access("./cache/", os.W_OK))  # Check if the directory is writable

True
True


## Using Embeddings with LocalFileStore (Persistent Storage)

Phương pháp được hỗ trợ chính để khởi tạo `CacheBackedEmbeddings` là `from_bytes_store`.

Nó chấp nhận các tham số sau:

-   `underlying_embeddings`: Trình nhúng được sử dụng để tạo `embeddings`.
-   `document_embedding_cache`: Một trong các triển khai `ByteStore` để lưu vào bộ nhớ cache `embeddings` của tài liệu.
-   `namespace`: (Tùy chọn, mặc định là `""`) Một namespace được sử dụng cho bộ nhớ cache tài liệu. Điều này được sử dụng để tránh xung đột với các bộ nhớ cache khác. Ví dụ: đặt nó thành tên của mô hình `embedding` đang được sử dụng.

**Lưu ý**: Điều quan trọng là phải đặt tham số `namespace` để tránh xung đột khi cùng một văn bản được nhúng bằng các mô hình `embedding` khác nhau.

Đầu tiên, hãy xem một ví dụ về việc lưu trữ `embeddings` bằng hệ thống tệp cục bộ và truy xuất chúng bằng kho vector FAISS.


In [6]:
from dotenv import load_dotenv
load_dotenv(override=True, dotenv_path="../.env")

True

In [7]:
from langchain.storage import LocalFileStore
from langchain_openai import OpenAIEmbeddings
from langchain.embeddings import CacheBackedEmbeddings
from langchain_community.vectorstores.faiss import FAISS

# Configure basic embeddings using OpenAI embeddings
underlying_embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# Set up a local file storage
store = LocalFileStore("./cache/")

# Create embeddings with caching support
cached_embedder = CacheBackedEmbeddings.from_bytes_store(
    underlying_embeddings=underlying_embeddings, 
    document_embedding_cache=store, 
    namespace=underlying_embeddings.model, # Create a cache-backed embedder using the base embedding and storage
)

In [8]:
# The cache is empty prior to embedding
list(store.yield_keys())

[]

Load the document, split it into chunks, embed each chunk and load it into the vector store.

In [9]:
from langchain.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter

raw_documents = TextLoader("./data/state_of_the_union.txt", encoding="utf-8").load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
documents = text_splitter.split_documents(raw_documents)

Create FAISS database from documents.

In [10]:
%time db = FAISS.from_documents(documents, cached_embedder)

CPU times: user 211 ms, sys: 25.8 ms, total: 237 ms
Wall time: 2.41 s


If we try to create the vector store again, it'll be much faster since it does not need to re-compute any embeddings.

In [11]:
# Create FAISS database using cached embeddings
%time db2 = FAISS.from_documents(documents, cached_embedder)

CPU times: user 23.7 ms, sys: 2.57 ms, total: 26.3 ms
Wall time: 25.5 ms


In [12]:
# Here are some of the embeddings that got created.
list(store.yield_keys())[:5]

['text-embedding-3-small81426526-23fe-58be-9e84-6c7c72c8ca9a',
 'text-embedding-3-small8061c36f-1bb7-5be5-9de2-afbed11ff5f2',
 'text-embedding-3-small280a9fbe-0d67-50df-9796-d781d74a0ea4',
 'text-embedding-3-small4e5734dd-6470-5d9e-a7b8-468ebc563c4e',
 'text-embedding-3-small9c15af4c-0e3b-5020-926f-752f448be568']

## Using `InMemoryByteStore` (Non-Persistent)

Để sử dụng một `ByteStore` khác, chỉ cần chỉ định `ByteStore` mong muốn khi tạo `CacheBackedEmbeddings`.

Dưới đây là một ví dụ về việc tạo đối tượng `embedding` được lưu vào bộ nhớ cache tương tự bằng cách sử dụng `InMemoryByteStore` không lưu trữ lâu dài.




In [13]:
from langchain.embeddings import CacheBackedEmbeddings
from langchain.storage import InMemoryByteStore

# Create an in-memory byte store
store = InMemoryByteStore()

underlying_embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

cached_embedder = CacheBackedEmbeddings.from_bytes_store(
    underlying_embeddings, store, namespace=underlying_embeddings.model
)

In [14]:
%time db = FAISS.from_documents(documents, cached_embedder)  
list(store.yield_keys())[:5]

CPU times: user 82 ms, sys: 3.17 ms, total: 85.2 ms
Wall time: 3.18 s


['text-embedding-3-small305efb5c-3f01-5657-bcf2-2b92fb1747ca',
 'text-embedding-3-small01dbc21f-5e4c-5fb5-8d13-517dbe7a32d4',
 'text-embedding-3-smalla5ef11e4-0474-5725-8d80-81c91943b37f',
 'text-embedding-3-small6d6cb8fc-721a-5a4c-bfe9-c83d2920c2bb',
 'text-embedding-3-small81426526-23fe-58be-9e84-6c7c72c8ca9a']